# 1.モデル用データ
- レースデータ
- 競走馬データ
- 血統データ
- 騎手データ
- 調教師データ
- 出馬表（予想対象）

レースデータと出馬表に競走馬データ以下の3つのデータを結合し。<br>
結合したデータで訓練したモデルを用いて出馬表の競走馬の<br>
が３着以内に入るかを予測する。

In [1]:
#ライブラリをインポート
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen
import optuna.integration.lightgbm as lgb_o
import warnings
from sklearn.impute import SimpleImputer


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [205]:
def confirm_data(df):
    display(df.info())
    display(df.head())

In [208]:
def confirm_series(df,column):
    display(df[column].value_counts())
    display(df[df[column].isnull()==True].head())

## 1.1　レースデータ

In [234]:
race_result = pd.read_pickle('data/race_results_11_20.pickle')

### 全体把握

In [235]:
race_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 444444 entries, 201101010101 to 202004040608
Data columns (total 24 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   着順              444444 non-null  object        
 1   枠番              444444 non-null  int64         
 2   馬番              444444 non-null  int64         
 3   馬名              444444 non-null  object        
 4   性齢              444444 non-null  object        
 5   斤量              444444 non-null  float64       
 6   騎手              444444 non-null  object        
 7   タイム             440759 non-null  object        
 8   着差              409540 non-null  object        
 9   単勝              444444 non-null  object        
 10  人気              442908 non-null  float64       
 11  馬体重             444444 non-null  object        
 12  調教師             444444 non-null  object        
 13  prize           444444 non-null  object        
 14  race_turn       444444 n

- タイム・着差・人気で欠損値が見られる

In [236]:
race_result.head()

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,prize,race_turn,course_len,weather,race_type,race_condition,date,race_park,horse_id,jockey_id,trainer_id
201101010101,1,2,2,オビーディエント,牡2,54.0,秋山真一,1:51.5,NaN,4.7,3.0,428(+8),[西] 加藤敬二,500.0,右,1800,晴,芝,良,2011-08-13,札幌,2009105006,01019,00379
201101010101,2,3,3,デビルシニスタ,牡2,54.0,岩田康誠,1:51.5,クビ,3.4,2.0,440(-14),[西] 友道康夫,500.0,右,1800,晴,芝,良,2011-08-13,札幌,2009100577,05203,01061
201101010101,3,5,5,アズマスターフィー,牡2,54.0,藤岡佑介,1:51.9,2.1/2,2.1,1.0,456(-6),[東] 武市康男,500.0,右,1800,晴,芝,良,2011-08-13,札幌,2009102908,01093,01089
201101010101,4,4,4,マスターズキング,牡2,54.0,吉田隼人,1:52.2,1.3/4,7.0,4.0,456(0),[西] 矢作芳人,500.0,右,1800,晴,芝,良,2011-08-13,札幌,2009109159,01095,01075
201101010101,5,1,1,コールミー,牝2,54.0,丸山元気,1:52.4,1.1/2,17.5,5.0,460(-10),[東] 鈴木伸尋,500.0,右,1800,晴,芝,良,2011-08-13,札幌,2009105263,01127,01026


- データ型にobjectが含まれているデータが多数あり、このままでは、MLできない。
- 着順や単勝といった数字で表されそうなデータもobjectと表示されており、原因を突き止める必要がある
- タイムや着差はレース開催後に判明するデータであるため、今回の予想には使用できない

### 個数の把握

In [237]:
race_result["着順"].value_counts()

1        27975
3        27967
2        27955
5        27950
6        27932
4        27929
7        27829
8        27584
9        26999
10       26002
11       24636
12       22999
13       20858
14       18696
15       16071
16       12982
1         3236
5         3232
4         3231
2         3228
3         3227
6         3223
7         3210
8         3171
9         3084
10        2954
17        2907
11        2785
12        2503
18        2425
13        2186
中         2149
14        1756
15        1394
取          771
除          765
16         319
17         252
失           10
2(降)         9
18           8
10(降)        6
3(降)         6
13(降)        6
6(降)         4
4(降)         4
12(降)        3
14(降)        3
11(降)        3
7(降)         2
9(降)         2
16(降)        2
15(降)        1
17(降)        1
12(再)        1
5(降)         1
Name: 着順, dtype: int64

- 競走中止や除外といった予想に使用できないデータがあるため、除外する必要がある
- 何故か1着が2つあるので（他の着順も同様）、修正する

In [238]:
race_result["枠番"].value_counts()

8    64220
7    62883
6    58775
5    57010
4    54644
3    52205
2    49060
1    45647
Name: 枠番, dtype: int64

- 出走頭数が多い場合6,7,8といった外枠の馬が増えるため、その枠のデータの個数が多くなっている

In [239]:
race_result["馬番"].value_counts()

2     31173
5     31173
4     31173
1     31173
3     31173
6     31147
7     31060
8     30822
9     30180
10    29130
11    27628
12    25837
13    23471
14    21032
15    17887
16    14474
17     3218
18     2693
Name: 馬番, dtype: int64

- 出走頭数はレースにより変動する。最小頭数は5頭であるため、5番まではデータ個数は同じである。それ以降は番号が増加するごとにデータ個数は減少している

In [240]:
race_result["性齢"].value_counts()

牡3     99977
牝3     81388
牡4     46636
牡2     41895
牝2     33323
牡5     32691
牝4     31003
牡6     19666
牝5     19067
牡7      9162
牝6      6532
セ5      3903
セ4      3825
セ3      3618
牡8      3250
セ6      2689
セ7      1481
牝7      1428
牡9       962
セ8       592
セ2       538
牝8       334
牡10      199
セ9       149
牡11       42
牝9        38
セ10       23
セ11       13
牝10        9
牡12        6
セ12        3
牡13        2
Name: 性齢, dtype: int64

- 性別と年齢が合体しているため、分解する必要がある

In [241]:
race_result["斤量"].value_counts()

54.0    125681
56.0     88868
55.0     80428
57.0     74853
53.0     23797
52.0     18472
51.0     14216
60.0      9081
58.0      3802
59.0      1729
50.0      1342
49.0       753
57.5       400
61.0       314
63.0       242
62.0       220
56.5        76
48.0        54
55.5        54
58.5        42
63.5        11
65.0         2
64.0         2
60.5         1
48.5         1
61.5         1
47.0         1
52.5         1
Name: 斤量, dtype: int64

In [242]:
race_result["単勝"].value_counts()

3.5      1854
4.1      1819
4.6      1818
3.8      1813
3.6      1808
3.7      1804
4.3      1785
3.9      1779
3.4      1778
4.2      1775
4.4      1760
4.0      1756
4.5      1752
2.9      1685
2.8      1676
3.3      1674
4.9      1673
4.8      1672
3.1      1669
4.7      1660
3.2      1656
3.0      1622
5.7      1608
5.1      1606
5.2      1597
5.3      1597
5.6      1593
2.7      1585
5.9      1584
5.5      1578
5.4      1545
6.3      1537
---      1536
2.6      1534
6.4      1533
6.0      1516
5.8      1515
5.0      1511
2.5      1502
6.1      1500
6.5      1481
6.2      1465
6.6      1450
7.0      1420
2.4      1407
6.8      1406
7.4      1384
7.1      1374
7.3      1359
7.2      1348
2.3      1341
6.9      1340
6.7      1317
7.5      1317
7.9      1276
7.6      1267
1.8      1266
1.9      1260
8.0      1245
1.7      1226
2.2      1219
7.7      1216
2.0      1210
7.8      1189
8.1      1186
8.3      1149
8.2      1136
2.1      1134
8.6      1125
8.5      1120
8.4      1097
8.7   

- 連続値であるため、データ個数が多い
- 単勝オッズが低いほど着順が良くなるように仮説が立てられる

In [243]:
race_result["人気"].value_counts()

5.0     31176
4.0     31174
3.0     31173
1.0     31173
2.0     31173
6.0     31141
7.0     31061
8.0     30806
9.0     30140
10.0    29079
11.0    27560
12.0    25729
13.0    23371
14.0    20879
15.0    17693
16.0    13834
17.0     3190
18.0     2556
Name: 人気, dtype: int64

- 整数だけなのに小数点型になっているので修正する
- 馬番と同じ理由で5を超えるとデータ個数は減少していく

In [244]:
race_result["馬体重"].value_counts()

470(0)      3002
460(0)      2810
468(0)      2797
472(0)      2734
462(0)      2726
466(0)      2706
480(0)      2674
476(0)      2668
464(0)      2631
458(0)      2619
456(0)      2610
474(0)      2590
478(0)      2577
454(0)      2470
450(0)      2455
452(0)      2396
484(0)      2316
482(0)      2307
486(0)      2241
448(0)      2225
490(0)      2215
444(0)      2141
488(0)      2114
446(0)      2105
442(0)      1965
440(0)      1957
492(0)      1862
438(0)      1726
494(0)      1685
496(0)      1674
500(0)      1668
436(0)      1594
468(-2)     1496
470(-2)     1479
498(0)      1475
434(0)      1471
476(-2)     1458
472(+2)     1451
460(-2)     1438
474(-2)     1428
470(+2)     1424
478(+2)     1412
432(0)      1398
480(-2)     1397
478(-2)     1392
468(+2)     1382
474(+2)     1379
464(-2)     1377
466(-2)     1365
502(0)      1365
458(-2)     1364
472(-2)     1352
466(+2)     1351
462(-2)     1349
480(+2)     1345
430(0)      1343
476(+2)     1340
460(+2)     1317
464(+2)     13

- 連続値に加えて増減があるため、データの個数が多い
- 馬体重と増減量に分割する必要がある

In [245]:
race_result["prize"].value_counts()

500.0       111389
750.0        44283
700.0        44033
720.0        21028
460.0        18840
510.0        16906
670.0        10613
760.0         9369
600.0         7051
1,050.0       6439
740.0         6034
1,100.0       4690
730.0         4642
480.0         4393
780.0         2984
970.0         2107
1,040.0       2086
990.0         2033
1,020.0       2014
400.0         1880
1,854.3       1351
1,853.6       1258
1,200.0       1176
1,350.0       1118
1,535.7       1112
790.0         1053
1,535.0       1046
1,855.0       1001
1,538.5        943
650.0          902
1,536.4        872
1,539.2        784
1,534.3        762
1,855.7        752
1,533.6        709
1,537.1        702
1,525.9        696
1,530.1        682
1,537.8        681
450.0          632
1,529.4        630
1,851.5        622
1,532.2        615
1,527.3        605
1,528.7        586
1,526.6        579
1,530.8        555
1,539.9        547
1,523.8        523
1,528.0        507
1,856.4        497
1,849.4        469
1,525.2     

- ネットでレース賞金も有効な変数として働くという記載があり追加
- レースレベルの代理変数にはなるか？

In [246]:
race_result["race_turn"].value_counts()

右    287038
左    139262
芝      8115
ー      6233
直      3796
Name: race_turn, dtype: int64

- "ー"というよくわからないデータがあるので、後ほど検証する
- "右回り"が多い

In [247]:
race_result["course_len"].value_counts()

1800    98861
1200    90237
1400    63942
1600    55483
2000    39639
1700    26196
1000    12457
2400     9976
2200     8280
2100     4532
1300     4057
1150     4048
1900     4025
2600     3852
3000     2940
1500     1886
2500     1706
2850     1496
2970     1332
2910     1269
2880     1135
2750      822
2890      820
3110      535
3170      503
3200      475
2770      455
2900      326
3300      322
3100      302
2300      227
3250      224
3140      214
2860      213
3330      193
3350      182
3290      165
3380      133
3400      130
4100      128
3900      121
3390      112
4250      111
3570      105
3930      101
3210       76
2800       69
3760       19
4260       12
Name: course_len, dtype: int64

- 1150や2770といった端数が多い→集約できないか？

In [248]:
race_result["weather"].value_counts()

晴     279780
曇     123027
雨      24945
小雨     15919
小雪       523
雪        250
Name: weather, dtype: int64

- 小雪や雪のような小数データは削除しても支障がないように思われる。
- そもそも天気は重要か？

In [249]:
race_result["race_type"].value_counts()

ダート    215185
芝      214911
障害      14348
Name: race_type, dtype: int64

- 種類が少ない
- 障害レースのデータ個数は相対的に見て少ない

In [250]:
race_result["race_condition"].value_counts()

良     313816
稍重     73162
重      37396
不良     20070
Name: race_condition, dtype: int64

In [251]:
race_result["race_park"].value_counts()

東京    69837
京都    68569
中山    64986
阪神    64958
新潟    44440
小倉    36024
中京    35158
福島    26112
函館    18344
札幌    16016
Name: race_park, dtype: int64

### データ修正

- 欠損値の補完を行う

#### 人気

In [252]:
race_result[race_result["人気"].isnull()==True].head()

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,prize,race_turn,course_len,weather,race_type,race_condition,date,race_park,horse_id,jockey_id,trainer_id
201101010103,取,2,2,クールダンス,牝3,54.0,藤岡佑介,NaN,NaN,---,NaN,計不,[東] 小西一男,480.0,右,1000,晴,ダート,良,2011-08-13,札幌,2008104698,01093,00405
201101010103,除,7,10,グランデアトレ,牡3,56.0,柴山雄一,NaN,NaN,---,NaN,460(-6),[東] 河野通文,480.0,右,1000,晴,ダート,良,2011-08-13,札幌,2008104383,05243,00404
201101010105,取,7,10,バブルジェット,牡2,54.0,藤岡佑介,NaN,NaN,---,NaN,計不,[東] 田島俊明,700.0,右,1500,晴,芝,良,2011-08-13,札幌,2009106045,01093,01112
201101010204,除,7,13,メジロサモン,牝3,54.0,藤岡佑介,NaN,NaN,---,NaN,418(+16),[東] 谷原義明,480.0,右,2000,曇,芝,良,2011-08-14,札幌,2008102532,01093,00402
201101010409,除,4,6,チャーリーポイント,牡4,57.0,藤田伸二,NaN,NaN,---,NaN,476(-4),[東] 杉浦宏昭,"1,087.8",右,1800,晴,芝,良,2011-08-21,札幌,2007106548,00705,01008


- 除外や出走取り消しとなった馬の場合のみ欠損値になる？

In [253]:
df = race_result[race_result["人気"].isnull()==True]
df["着順"].value_counts()

取    771
除    765
Name: 着順, dtype: int64

- 予想通りの結果であったため、着順のデータ前処理の段階で対応する

#### 着順
- 数字が含まれているデータについては数字部分だけを取り出す
- 含まれていないデータについては削除する

- replace関数を用いて()を空白に変換する。ただしobject型でないデータに対しては一括で変換できないため、
- まずはobject型に変換する

In [254]:
#object型に変換し、置換する
race_result["着順"] = race_result["着順"].astype(str)
race_result["着順"] = race_result["着順"].map(lambda x : x.replace("(降)",""))
race_result["着順"] = race_result["着順"].map(lambda x : x.replace("(再)",""))

In [255]:
#正しく置換されているか確認
race_result["着順"].value_counts()

1     31211
3     31200
2     31192
5     31183
4     31164
6     31159
7     31041
8     30755
9     30085
10    28962
11    27424
12    25506
13    23050
14    20455
15    17466
16    13303
17     3160
18     2433
中      2149
取       771
除       765
失        10
Name: 着順, dtype: int64

In [256]:
#数値に変換できるものは変換し、変換できないものについては欠損値を返す
race_result["着順"] = pd.to_numeric(race_result["着順"],errors="coerce")
#欠損値となったレコードを削除する
race_result.dropna(subset=["着順"],inplace=True) 
#整数型に置換
race_result["着順"] = race_result["着順"].astype(int)

In [257]:
#確認
race_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 440749 entries, 201101010101 to 202004040608
Data columns (total 24 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   着順              440749 non-null  int64         
 1   枠番              440749 non-null  int64         
 2   馬番              440749 non-null  int64         
 3   馬名              440749 non-null  object        
 4   性齢              440749 non-null  object        
 5   斤量              440749 non-null  float64       
 6   騎手              440749 non-null  object        
 7   タイム             440749 non-null  object        
 8   着差              409540 non-null  object        
 9   単勝              440749 non-null  object        
 10  人気              440749 non-null  float64       
 11  馬体重             440749 non-null  object        
 12  調教師             440749 non-null  object        
 13  prize           440749 non-null  object        
 14  race_turn       440749 n

- 着順が整数型になっていることを確認
- 人気の欠損値もない

#### 性齢
- 性別と年齢に分ける
- 1文字目が性別,２文字目以降が年齢なのでスライスとmap関数を使って分割する

In [258]:
#分割する。年齢は数値型に変換する
race_result["性"] = race_result["性齢"].map(lambda x: str(x)[0])
race_result["年齢"] = race_result["性齢"].map(lambda x: str(x)[1:]).astype(int)

In [259]:
#確認
race_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 440749 entries, 201101010101 to 202004040608
Data columns (total 26 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   着順              440749 non-null  int64         
 1   枠番              440749 non-null  int64         
 2   馬番              440749 non-null  int64         
 3   馬名              440749 non-null  object        
 4   性齢              440749 non-null  object        
 5   斤量              440749 non-null  float64       
 6   騎手              440749 non-null  object        
 7   タイム             440749 non-null  object        
 8   着差              409540 non-null  object        
 9   単勝              440749 non-null  object        
 10  人気              440749 non-null  float64       
 11  馬体重             440749 non-null  object        
 12  調教師             440749 non-null  object        
 13  prize           440749 non-null  object        
 14  race_turn       440749 n

- 年齢は数値型になっている

#### 単勝
- 数値型に変換する
- 着順とは異なり、数字以外の要素に何が入っているかわからないので、置換できるものは置換してそうでないものは一旦放置する

In [260]:
race_result["単勝"] = pd.to_numeric(race_result["単勝"],errors="ignore")

In [261]:
#個数を確認
race_result["単勝"].value_counts()

3.5      1914
4.6      1906
4.1      1897
3.8      1894
3.7      1881
3.6      1879
4.0      1864
3.9      1862
4.2      1862
3.4      1859
4.3      1859
4.4      1841
4.5      1832
2.9      1774
4.8      1763
2.8      1756
3.1      1748
3.3      1746
3.2      1744
4.7      1742
4.9      1735
3.0      1703
5.1      1676
5.7      1675
2.7      1672
5.3      1667
5.2      1664
5.9      1662
5.6      1653
5.5      1643
5.4      1611
6.3      1602
2.6      1601
6.4      1591
6.0      1590
2.5      1584
5.8      1584
5.0      1575
6.1      1560
6.5      1546
6.2      1514
6.6      1511
7.0      1488
2.4      1479
6.8      1474
7.4      1433
7.3      1416
7.1      1414
7.2      1411
6.9      1409
2.3      1398
6.7      1385
7.5      1382
7.9      1327
7.6      1323
1.9      1315
1.8      1314
8.0      1302
1.7      1288
2.2      1274
7.7      1261
2.0      1261
7.8      1245
8.1      1239
2.1      1192
8.3      1189
8.2      1188
8.6      1169
8.5      1163
8.4      1142
8.7      1124
9.0   

In [262]:
#確認
race_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 440749 entries, 201101010101 to 202004040608
Data columns (total 26 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   着順              440749 non-null  int64         
 1   枠番              440749 non-null  int64         
 2   馬番              440749 non-null  int64         
 3   馬名              440749 non-null  object        
 4   性齢              440749 non-null  object        
 5   斤量              440749 non-null  float64       
 6   騎手              440749 non-null  object        
 7   タイム             440749 non-null  object        
 8   着差              409540 non-null  object        
 9   単勝              440749 non-null  float64       
 10  人気              440749 non-null  float64       
 11  馬体重             440749 non-null  object        
 12  調教師             440749 non-null  object        
 13  prize           440749 non-null  object        
 14  race_turn       440749 n

- 小数点型になっており、欠損値もないため問題なさそう

#### 人気（型変換）
- 何故か小数点型になっているので整数型に変換する

In [263]:
race_result["人気"] = race_result["人気"].astype(int)

In [264]:
#確認
race_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 440749 entries, 201101010101 to 202004040608
Data columns (total 26 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   着順              440749 non-null  int64         
 1   枠番              440749 non-null  int64         
 2   馬番              440749 non-null  int64         
 3   馬名              440749 non-null  object        
 4   性齢              440749 non-null  object        
 5   斤量              440749 non-null  float64       
 6   騎手              440749 non-null  object        
 7   タイム             440749 non-null  object        
 8   着差              409540 non-null  object        
 9   単勝              440749 non-null  float64       
 10  人気              440749 non-null  int64         
 11  馬体重             440749 non-null  object        
 12  調教師             440749 non-null  object        
 13  prize           440749 non-null  object        
 14  race_turn       440749 n

- 整数型に変換されている

#### 馬体重
- 馬体重と増減値に分解する

In [265]:
#str.split("(")とすることで(で単語列を分解する
#引数をexpand=Trueとすることで別々の列に分解できる
race_result["馬体重"].str.split("(",expand=True).head()

,0,1
201101010101,428,+8)
201101010101,440,-14)
201101010101,456,-6)
201101010101,456,0)
201101010101,460,-10)


In [266]:
#0列目を整数に変換し、新しい列を作成する
#”計不”という列が1つだけあったので削除する
race_result["馬体重"] = race_result["馬体重"].replace('計不', np.nan)
race_result.dropna(subset=["馬体重"],inplace=True) 
race_result["体重"] = race_result["馬体重"].str.split("(",expand=True)[0].astype(int)
#1列目については)も外し、増減値という列を作成する
race_result["増減"] = race_result["馬体重"].str.split("(",expand=True)[1].str.split(")",expand=True)[0]

In [267]:
race_result["増減"] = race_result["増減"].astype(int) 

### カラム名変更・データ抽出

In [269]:
# カラム名変換
race_result.rename(columns={"枠番": "frame_number", "馬番": "horse_number","斤量":"weight_j" ,"単勝": "odds",
                   "人気": "favorite","性":"sex","年齢":"age","体重":"weight","増減":"weight_change"}, inplace=True)

In [271]:
race_result.rename(columns={"着順":"rank"},inplace=True)

In [273]:
# 不要な列を削除
race_result.drop(["馬名", "性齢", "騎手", "タイム","着差", "馬体重", "調教師"], axis=1, inplace=True)

In [274]:
#確認
race_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 440748 entries, 201101010101 to 202004040608
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   rank            440748 non-null  int64         
 1   frame_number    440748 non-null  int64         
 2   horse_number    440748 non-null  int64         
 3   weight_j        440748 non-null  float64       
 4   odds            440748 non-null  float64       
 5   favorite        440748 non-null  int64         
 6   prize           440748 non-null  object        
 7   race_turn       440748 non-null  object        
 8   course_len      440748 non-null  int64         
 9   weather         440748 non-null  object        
 10  race_type       440748 non-null  object        
 11  race_condition  440748 non-null  object        
 12  date            440748 non-null  datetime64[ns]
 13  race_park       440748 non-null  object        
 14  horse_id        440748 n

## 1.2　競走馬データ

In [2]:
horse_result = pd.read_pickle('data/horse_results_all.pickle')

### 全体把握

In [3]:
horse_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 770638 entries, 2013102966 to 2017105663
Data columns (total 31 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   日付          770638 non-null  object 
 1   開催          770627 non-null  object 
 2   天気          768071 non-null  object 
 3   R           769920 non-null  float64
 4   レース名        770638 non-null  object 
 5   映像          0 non-null       float64
 6   頭数          770503 non-null  float64
 7   枠番          769892 non-null  float64
 8   馬番          770638 non-null  int64  
 9   オッズ         762037 non-null  float64
 10  人気          762540 non-null  float64
 11  着順          767312 non-null  object 
 12  騎手          770590 non-null  object 
 13  斤量          770575 non-null  float64
 14  距離          770638 non-null  object 
 15  馬場          768685 non-null  object 
 16  馬場指数        749539 non-null  object 
 17  タイム         759707 non-null  object 
 18  着差          759223 non-null  float64

In [4]:
horse_result.head()

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,人気,着順,騎手,斤量,距離,馬場,馬場指数,タイム,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金,baba_index,time_index,bikou
2013102966,2017/01/26,大井,晴,10.0,白富士特別競走,NaN,16.0,8.0,15,101.9,15.0,16,吉井竜一,56.0,ダ1600,良,**,1:47.7,3.8,**,16-16-16-16,37.8-41.0,42.0,460(-7),NaN,NaN,ケイズイーグル,NaN,10,26,
2013102966,2017/01/12,船橋,晴,9.0,C1二,NaN,9.0,8.0,8,116.1,7.0,8,柏木健宏,56.0,ダ1600,重,**,1:50.5,4.5,**,8-8-8-8,40.8-38.5,41.3,467(-15),NaN,NaN,ハクサンワールド,NaN,12,12,
2013102966,2016/12/29,大井,晴,7.0,冬野特別競走,NaN,16.0,6.0,11,183.6,16.0,16,吉井竜一,56.0,ダ1200,重,**,1:20.4,6.1,**,16-16,35.1-39.2,42.4,482(+1),NaN,NaN,クリルカレント,NaN,10,-8,
2013102966,2016/12/15,川崎,晴,12.0,フロイデ特別,NaN,9.0,7.0,8,88.3,9.0,8,柏木健宏,56.0,ダ1500,不,**,1:39.2,1.9,**,8-8-9-9,31.4-39.8,40.5,481(+1),NaN,NaN,デルマハンニャ,NaN,13,60,
2013102966,2016/11/29,大井,晴,10.0,初冬特別競走,NaN,14.0,4.0,5,80.0,11.0,14,矢野貴之,56.0,ダ1600,重,**,1:49.9,5.5,**,7-7-8-10,38.2-40.9,45.5,480(-8),NaN,NaN,ツナグテ,NaN,10,4,


分析に使用するデータのみを抽出する

In [112]:
horse_result = horse_result[["日付","開催","オッズ","人気","着順","斤量","距離","通過","上り","賞金","time_index"]] 

In [113]:
horse_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 758531 entries, 2013102966 to 2017105663
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   日付          758531 non-null  object 
 1   開催          758531 non-null  object 
 2   オッズ         758531 non-null  float64
 3   人気          758531 non-null  float64
 4   着順          758531 non-null  object 
 5   斤量          758531 non-null  float64
 6   距離          758531 non-null  object 
 7   通過          758531 non-null  object 
 8   上り          758531 non-null  float64
 9   賞金          758531 non-null  float64
 10  time_index  758531 non-null  object 
dtypes: float64(5), object(6)
memory usage: 69.4+ MB


In [114]:
horse_result.head()

,日付,開催,オッズ,人気,着順,斤量,距離,通過,上り,賞金,time_index
2013102966,2017/01/26,大井,101.9,15.0,16,56.0,ダ1600,16-16-16-16,42.0,0.0,26
2013102966,2017/01/12,船橋,116.1,7.0,8,56.0,ダ1600,8-8-8-8,41.3,0.0,12
2013102966,2016/12/29,大井,183.6,16.0,16,56.0,ダ1200,16-16,42.4,0.0,-8
2013102966,2016/12/15,川崎,88.3,9.0,8,56.0,ダ1500,8-8-9-9,40.5,0.0,60
2013102966,2016/11/29,大井,80.0,11.0,14,56.0,ダ1600,7-7-8-10,45.5,0.0,4


欠損値が存在する行が多い

### 個数の把握・欠損値の確認

In [115]:
horse_result["開催"].value_counts()

園田       55003
高知       42172
名古屋      41765
佐賀       39798
金沢       27875
大井       27336
笠松       25347
水沢       20496
盛岡       20359
川崎       19095
門別       17341
船橋       16331
浦和       15946
姫路        4038
2東京12     1507
2新潟6      1501
5東京7      1474
2阪神6      1459
3東京4      1458
5阪神7      1456
3新潟2      1454
3東京2      1451
1小倉1      1447
1東京8      1441
1東京5      1436
2新潟4      1436
2東京9      1435
1東京6      1433
1中山1      1432
2東京3      1432
1小倉2      1431
4東京2      1431
1中山5      1430
3新潟4      1429
2阪神7      1428
2東京10     1426
2東京11     1423
3新潟1      1423
1東京4      1423
4中山8      1420
4東京1      1419
4中山3      1418
2中京6      1417
1東京7      1417
1小倉4      1416
2東京7      1413
2新潟1      1411
2阪神5      1410
2東京2      1410
4中山4      1410
1中山3      1406
3中山8      1406
1小倉7      1405
1中山4      1404
1東京2      1404
4阪神6      1403
2新潟5      1401
5東京5      1399
2中山8      1399
4中山6      1399
3東京3      1397
1中山6      1396
1小倉8      1396
1小倉6      1395
2中京3      1391
2新潟3      1389
5阪神8      

- 中央競馬は回＋競馬場＋レース番号という組み合わせになっている
- 地方競馬は競馬場がそのまま
- 形を揃える必要がある

In [12]:
horse_result[horse_result["開催"].isnull()]

,日付,開催,オッズ,人気,着順,斤量,距離,通過,ペース,上り,馬体重,賞金,time_index
2013106119,2020/02/29,NaN,34.0,11.0,6,57.0,ダ1800,NaN,NaN,NaN,計不,NaN,
2014106191,2020/02/29,NaN,1.4,1.0,2,55.5,芝2100,NaN,NaN,NaN,計不,NaN,
2014110060,2021/02/20,NaN,NaN,1.0,2,57.0,ダ1200,NaN,NaN,NaN,計不,NaN,
2014110060,2020/02/29,NaN,11.0,5.0,2,57.0,ダ1200,NaN,NaN,NaN,計不,NaN,
2015110078,2021/02/20,NaN,NaN,5.0,1,57.0,ダ1200,NaN,NaN,NaN,計不,NaN,
2015104879,2021/02/20,NaN,NaN,5.0,9,57.0,ダ1800,NaN,NaN,NaN,計不,NaN,
2013190003,2018/09/23,NaN,NaN,1.0,1,59.0,芝2400,NaN,NaN,NaN,計不,NaN,
2016104458,2020/02/29,NaN,9.0,4.0,7,57.3,ダ1800,NaN,NaN,NaN,計不,NaN,
2017110117,2020/02/29,NaN,17.0,7.0,1,55.0,ダ1600,NaN,NaN,NaN,計不,NaN,
2018104234,2021/02/20,NaN,NaN,7.0,1,57.0,ダ1600,NaN,NaN,NaN,計不,NaN,


- 調べたところ、海外の一部レースの模様
- 個数も少ないので削除する

In [11]:
horse_result["オッズ"].value_counts()

1.7       4676
1.8       4490
1.6       4389
1.9       4386
1.5       4284
2.8       4255
2.5       4201
2.0       4188
3.0       4184
2.7       4157
2.4       4157
3.1       4146
3.5       4143
3.4       4091
2.9       4078
2.2       4038
2.6       4009
1.4       4009
2.3       4003
3.3       3989
3.2       3983
3.8       3945
2.1       3928
3.6       3917
3.9       3911
3.7       3881
4.1       3851
4.2       3834
4.0       3820
4.6       3723
4.3       3722
4.5       3712
4.4       3665
1.3       3608
4.7       3597
4.8       3547
4.9       3461
5.1       3290
5.0       3260
5.2       3259
5.4       3217
5.3       3191
1.2       3152
5.6       3112
5.5       3091
5.7       3053
5.9       3053
5.8       3052
6.0       3049
6.1       3020
6.3       2907
6.2       2898
6.5       2836
6.4       2825
6.6       2739
6.9       2684
6.7       2682
7.0       2676
1.1       2647
7.2       2633
6.8       2627
7.1       2614
7.4       2567
7.3       2530
7.5       2445
7.6       2345
7.7       

In [15]:
horse_result[horse_result["オッズ"].isnull()].head()

,日付,開催,オッズ,人気,着順,斤量,距離,通過,ペース,上り,馬体重,賞金,time_index
2013106165,2016/12/19,浦和,NaN,NaN,除,55.0,ダ1400,NaN,37.1-40.4,NaN,487(+6),NaN,0
2012105141,2016/02/24,園田,NaN,NaN,取,54.0,ダ1230,NaN,NaN,NaN,計不,NaN,0
2012100183,2015/12/05,佐賀,NaN,NaN,除,53.0,ダ1400,NaN,NaN,NaN,426(+1),NaN,0
2012105211,2015/10/08,門別,NaN,NaN,NaN,54.0,ダ1000,NaN,NaN,NaN,計不,NaN,0
2012103208,2018/07/17,門別,NaN,NaN,取,56.0,ダ1700,NaN,0.0-41.2,NaN,計不,NaN,0


- レースデータと同じ理由で欠損値になっている。そのまま削除。

In [18]:
horse_result["人気"].value_counts()

1.0     81293
2.0     75357
3.0     71112
4.0     68110
5.0     65555
6.0     63069
7.0     60114
8.0     56267
9.0     50534
10.0    42565
11.0    33675
12.0    27423
13.0    20520
14.0    17853
15.0    14205
16.0    10768
17.0     2326
18.0     1793
0.0         1
Name: 人気, dtype: int64

In [19]:
horse_result[horse_result["人気"]==0]

,日付,開催,オッズ,人気,着順,斤量,距離,通過,ペース,上り,馬体重,賞金,time_index
2014103283,2019/03/30,オースト,0.0,0.0,取,54.0,芝1600,0-0,NaN,NaN,計不,NaN,


- 海外競馬なので削除する

In [67]:
horse_result[horse_result["人気"].isnull()].head()

,日付,開催,オッズ,人気,着順,斤量,距離,通過,上り,馬体重,賞金,time_index,race_park


- オッズと同じ。そのまま削除。

In [167]:
horse_result["着順"].value_counts()

1        55633
2        51940
3        49885
4        47866
5        46572
6        45071
7        43662
8        41254
9        37214
10       31833
11       25089
12       20771
1        20054
2        18441
3        17927
4        17366
5        17079
6        16751
7        16170
13       15701
8        15352
14       13699
9        13658
10       11151
15       11097
11        8403
16        8272
12        6547
13        4301
14        3666
15        2940
16        2103
17        1878
18        1374
17         410
18         327
失           11
2(降)        11
3(降)         6
10(降)        5
4(降)         3
14(降)        2
6(降)         2
5(降)         2
11(降)        1
12(降)        1
9(降)         1
13(降)        1
Name: 着順, dtype: int64

In [169]:
horse_result[horse_result["着順"].isnull()].head()

,日付,開催,オッズ,人気,着順,斤量,距離,通過,上り,賞金,time_index,race_park,race_type,course_len,final_corner


降着について同じ要領で処理する

In [22]:
horse_result["斤量"].value_counts()

54.0    254083
56.0    248519
55.0     93227
57.0     78514
53.0     36055
52.0     23619
51.0     16889
60.0      7823
58.0      3820
50.0      2537
59.0      1499
56.5       940
49.0       773
54.5       657
57.5       487
61.0       276
55.5       250
62.0       196
63.0       181
58.5        52
48.0        51
53.5        42
59.5        31
52.5        25
60.5         8
51.5         6
50.5         4
47.0         3
62.5         2
64.0         2
48.5         1
49.5         1
61.5         1
57.3         1
Name: 斤量, dtype: int64

In [23]:
horse_result[horse_result["斤量"].isnull()].head()

,日付,開催,オッズ,人気,着順,斤量,距離,通過,ペース,上り,馬体重,賞金,time_index
2010104296,2015/10/17,オースト,NaN,5.0,6,NaN,芝2400,NaN,NaN,NaN,計不,NaN,
2009105961,2015/06/17,イギリス,NaN,3.0,6,NaN,芝2000,NaN,NaN,NaN,計不,NaN,
2011190001,2015/03/28,イギリス,NaN,3.0,4,NaN,ダ2200,NaN,NaN,NaN,計不,NaN,
2011190001,2014/09/16,イギリス,NaN,5.0,4,NaN,芝2300,NaN,NaN,NaN,計不,NaN,
2011190001,2014/09/06,イギリス,NaN,10.0,14,NaN,芝2400,NaN,NaN,NaN,計不,NaN,


- 海外競馬なので削除する

In [24]:
horse_result["距離"].value_counts()

ダ1400    213576
ダ1200     70738
ダ1800     61600
ダ1600     59359
ダ1300     44407
芝1200     37900
芝1600     36285
芝2000     34413
芝1800     33814
ダ1700     32093
ダ1500     30211
芝1400     22776
ダ1000     11127
ダ1230      7925
芝2200      7283
芝2400      6932
ダ1900      5367
ダ2000      4978
ダ900       4770
ダ2100      4698
ダ1150      4075
芝1000      3935
芝2600      3647
ダ800       3515
障3000      2147
ダ2400      2080
ダ1870      2037
ダ1750      2002
芝1500      1810
芝2500      1282
障2850      1200
障2750      1055
ダ850       1013
障2970       904
芝1700       868
障2890       832
障2880       675
障2910       579
障2860       571
障2770       526
ダ2500       420
障3110       401
障3170       369
ダ820        359
芝3000       350
障3300       292
ダ2200       271
障3330       225
芝2300       217
障3200       206
障3250       197
障3350       182
障3290       171
障3100       171
障3380       161
芝3200       161
障3140       157
障3390       153
芝3400       118
障2900       113
ダ2600       105
芝3600       104
障4100   

In [26]:
horse_result[horse_result["距離"].isnull()]

,日付,開催,オッズ,人気,着順,斤量,距離,通過,ペース,上り,馬体重,賞金,time_index


In [27]:
horse_result["通過"].value_counts()

1-1-1-1        25114
2-2            18403
1-1            18007
2-2-2-2        14256
3-3            12989
4-4             9132
5-5             8122
1-1-1           7944
7-7             7656
6-6             6944
8-8             6595
3-3-3-3         6431
9-9             6101
10-10           5738
11-11           5320
1-1-1-2         4918
12-12           4842
2-2-2           4606
2-2-2-1         4478
13-13           4336
14-14           4041
4-3             3978
5-4             3958
15-15           3949
8-7             3872
6-5             3676
7-6             3642
3-2             3544
16-16           3520
3-4             3495
2-2-1-1         3465
9-8             3352
10-10-10-10     3229
2-2-2-3         3202
4-5             3178
9-9-9-9         3100
3-3-3-2         3099
10-9            3026
7-8             3023
6-7             3018
5-6             3007
4-4-4-4         2978
3-3-2-2         2965
8-8-8-8         2960
5-5-5-5         2881
11-10           2866
7-7-7-7         2740
8-9          

- このままでは使用できない
- 最終コーナーの順位だけ採用する

In [29]:
horse_result[horse_result["通過"].isnull()].head()

,日付,開催,オッズ,人気,着順,斤量,距離,通過,ペース,上り,馬体重,賞金,time_index
2013106165,2016/12/19,浦和,NaN,NaN,除,55.0,ダ1400,NaN,37.1-40.4,NaN,487(+6),NaN,0
2012105141,2016/02/24,園田,NaN,NaN,取,54.0,ダ1230,NaN,NaN,NaN,計不,NaN,0
2012100183,2015/12/05,佐賀,NaN,NaN,除,53.0,ダ1400,NaN,NaN,NaN,426(+1),NaN,0
2012105211,2015/10/08,門別,NaN,NaN,NaN,54.0,ダ1000,NaN,NaN,NaN,計不,NaN,0
2012103208,2018/07/17,門別,NaN,NaN,取,56.0,ダ1700,NaN,0.0-41.2,NaN,計不,NaN,0


　中止が多い？ 念のため着順を確認する

In [30]:
horse_result[horse_result["通過"].isnull()]["着順"].value_counts()

取     2885
除     1839
中      853
1      151
2       93
4       73
3       64
5       57
6       48
7       27
8       27
9       25
10      23
11      15
14      13
1       11
12      11
3       11
8       10
5        8
13       8
6        7
4        6
2        6
15       4
9        4
7        4
11       3
10       3
17       3
12       2
14       2
16       2
23       1
13       1
Name: 着順, dtype: int64

着順が入っているレコードが気になる

In [45]:
horse_result.query('通過!=通過 and 着順==1', engine='numexpr').head()

,日付,開催,オッズ,人気,着順,斤量,距離,通過,上り,馬体重,賞金,time_index
2011110078,2015/06/07,韓国,NaN,1.0,1,57.0,ダ1400,NaN,NaN,計不,NaN,
2012104889,2016/03/26,アラブ首,NaN,4.0,1,57.0,芝1800,NaN,NaN,計不,NaN,
2011104201,2016/11/24,アメリカ,NaN,3.0,1,55.5,芝2200,NaN,NaN,計不,NaN,
2012110091,2018/09/09,韓国,NaN,1.0,1,57.0,ダ1200,NaN,NaN,525,NaN,
2012104668,2016/12/11,シャティ,11.4,4.0,1,57.0,芝2400,NaN,NaN,計不,NaN,


海外レースのため削除する

In [46]:
horse_result["上り"].value_counts()

39.6    12468
39.4    12455
39.5    12356
39.7    12264
39.9    12237
40.0    12151
39.8    12123
39.3    12113
39.2    12079
39.1    12009
40.1    11824
40.2    11776
39.0    11749
38.9    11635
40.3    11417
38.8    11358
38.7    11101
38.6    11045
40.4    10953
38.5    10764
40.5    10734
38.4    10636
38.3    10329
40.6    10288
38.2     9996
40.7     9850
38.1     9660
40.8     9573
38.0     9356
37.9     9218
40.9     9117
37.8     8969
41.0     8906
37.7     8783
37.6     8526
41.1     8299
37.5     8156
37.3     8014
37.4     7985
37.2     7713
41.2     7687
37.1     7624
37.0     7528
41.3     7454
36.8     7255
36.9     7216
36.6     6994
36.5     6925
36.4     6898
35.5     6877
36.7     6871
41.4     6855
36.2     6847
36.1     6833
36.3     6819
35.9     6794
36.0     6777
35.7     6775
35.4     6769
35.6     6749
35.8     6747
35.3     6662
41.5     6650
35.2     6562
35.0     6506
35.1     6372
34.9     6102
41.6     6078
34.8     5929
41.7     5767
34.7     5658
34.6  

In [47]:
horse_result[horse_result["上り"].isnull()].head()

,日付,開催,オッズ,人気,着順,斤量,距離,通過,上り,馬体重,賞金,time_index
2013106165,2016/12/19,浦和,NaN,NaN,除,55.0,ダ1400,NaN,NaN,487(+6),NaN,0
2012105756,2016/07/25,川崎,31.9,6.0,中,54.0,ダ900,4-6,NaN,482(+3),NaN,0
2012105470,2015/08/30,2札幌4,218.4,12.0,中,54.0,ダ1700,3-3-3,NaN,478(0),NaN,0
2012105141,2017/06/11,金沢,62.4,9.0,中,52.0,ダ1400,1-1,NaN,450(+2),NaN,0
2012105141,2016/02/24,園田,NaN,NaN,取,54.0,ダ1230,NaN,NaN,計不,NaN,0


他の列でも異常値を観測していると思われるので他の列を削除後に確認する

In [54]:
horse_result["賞金"].value_counts()

75.0       10373
110.0      10228
200.0       9903
130.0       9033
50.0        8562
500.0       7341
20.0        6275
30.0        5662
300.0       5510
190.0       5418
180.0       5290
25.0        4454
3.0         4180
2.0         4096
1.0         4004
150.0       3748
4.0         3668
750.0       3655
70.0        3611
1.8         3592
280.0       3491
10.0        3422
100.0       3391
12.0        3259
2.4         3144
700.0       3129
35.0        3040
5.0         3011
2.8         2917
40.0        2804
120.0       2791
1.2         2771
1.5         2768
6.0         2650
2.5         2575
1.4         2362
15.0        2273
7.0         2218
14.0        2194
3.5         2190
18.0        2160
230.0       2123
51.0        2111
1.6         2069
27.0        2069
60.0        2053
510.0       2006
77.0        2006
8.4         1985
2.2         1977
16.0        1974
270.0       1962
80.0        1936
7.5         1919
0.8         1879
4.2         1852
160.0       1805
1.3         1805
2.6         18

In [56]:
horse_result[horse_result["賞金"].isnull()].head(30)

,日付,開催,オッズ,人気,着順,斤量,距離,通過,上り,馬体重,賞金,time_index
2013102966,2017/01/26,大井,101.9,15.0,16,56.0,ダ1600,16-16-16-16,42.0,460(-7),NaN,26
2013102966,2017/01/12,船橋,116.1,7.0,8,56.0,ダ1600,8-8-8-8,41.3,467(-15),NaN,12
2013102966,2016/12/29,大井,183.6,16.0,16,56.0,ダ1200,16-16,42.4,482(+1),NaN,-8
2013102966,2016/12/15,川崎,88.3,9.0,8,56.0,ダ1500,8-8-9-9,40.5,481(+1),NaN,60
2013102966,2016/11/29,大井,80.0,11.0,14,56.0,ダ1600,7-7-8-10,45.5,480(-8),NaN,4
2013102966,2016/10/16,4東京5,90.5,15.0,15,55.0,ダ1300,9-10,38.0,488(-2),NaN,64
2013102966,2016/09/24,4中山5,199.1,15.0,12,55.0,ダ1200,13-12,37.0,490(+18),NaN,81
2013102966,2016/05/08,2東京6,233.1,11.0,11,56.0,芝1600,6-6,36.3,472(+2),NaN,59
2013102966,2016/04/09,3中山5,151.7,14.0,16,56.0,ダ1200,9-10,39.0,470(0),NaN,55
2013102966,2016/03/27,3中山2,79.2,13.0,16,56.0,芝1200,14-15,34.8,470(-4),NaN,78


賞金がもらえる着順以外は欠損値になっているので0で埋める

In [136]:
horse_result["time_index"].value_counts()

                                                                                                                                                  16972
51                                                                                                                                                 9380
82                                                                                                                                                 9369
83                                                                                                                                                 9326
81                                                                                                                                                 9306
86                                                                                                                                                 9258
79                                                                                      

In [105]:
horse_result[horse_result["time_index"].isnull()].head(30)

,日付,開催,オッズ,人気,着順,斤量,距離,通過,上り,賞金,time_index
2019106008,2021/07/03,1福島1,8.2,5.0,11,51.0,ダ1150,13-12,37.0,0.0,NaN
2019106700,2021/07/18,1函館6,298.6,12.0,11,52.0,芝1200,9-9,35.5,0.0,NaN
2019106700,2021/07/03,1函館1,143.5,10.0,9,52.0,芝1200,10-10,36.8,0.0,NaN
2019101479,2021/07/17,1函館5,6.3,2.0,4,54.0,芝1800,1-1-1-1,36.6,77.0,NaN
2019101479,2021/07/04,1函館2,21.4,5.0,4,54.0,芝1800,1-1-1-1,36.6,110.0,NaN
2019100800,2021/07/25,1函館8,6.6,4.0,3,54.0,芝1800,5-5-5-4,38.2,180.0,NaN
2019102459,2021/07/11,3小倉4,8.0,4.0,4,54.0,芝1800,2-2-2-2,35.4,110.0,NaN
2019102433,2021/06/27,3東京8,72.4,12.0,14,54.0,ダ1400,10-10,38.8,0.0,NaN
2018103990,2021/07/04,1福島2,24.7,9.0,11,54.0,芝1200,12-11,36.1,0.0,NaN
2018103990,2021/04/24,1新潟5,137.6,13.0,14,54.0,ダ1200,13-11,38.2,0.0,NaN


欠損値が多く見られる

### データ修正

#### 開催
競馬場だけになるようにデータをクレンジングする

In [116]:
horse_result["race_park"] = horse_result["開催"].str.extract(r"(\D+)")[0]

欠損値を削除する

In [117]:
horse_result[horse_result["race_park"].isnull()].head()

,日付,開催,オッズ,人気,着順,斤量,距離,通過,上り,賞金,time_index,race_park


In [118]:
horse_result.dropna(subset=["race_park"], inplace=True)

####　　オッズ・人気・斤量
欠損値を削除する

In [119]:
horse_result.dropna(subset=["オッズ"], inplace=True)

In [120]:
horse_result.dropna(subset=["人気"], inplace=True)

In [121]:
horse_result.dropna(subset=["斤量"], inplace=True)

#### 着順
降着を処理する

In [171]:
horse_result["着順"] = horse_result["着順"].astype(str)
horse_result["着順"] = horse_result["着順"].map(lambda x: x.replace("(降)", ""))
horse_result["着順"] = pd.to_numeric(horse_result["着順"], errors="coerce")
horse_result.dropna(subset=["着順"], inplace=True)

####　　距離
レースタイプと距離に分離する

In [122]:
horse_result["race_type"] = horse_result.loc[:, "距離"].str[0]

In [123]:
horse_result["course_len"] = horse_result["距離"].apply(lambda x: str(x)[1:]).astype(float) // 100

####　　通過
最終コーナーの値だけ抜き出す
関数を作成して適用する

In [124]:
def corner(x, n):
    if type(x) != str:  # xが文字列型でなかった場合、ｘをそのまま返す
        return x
    elif n == 4:
        return int(re.findall(r'\d+', x)[-1])
    elif n == 1:
        return int(re.findall(r'\d+', x)[0])

In [125]:
horse_result['final_corner'] = horse_result['通過'].map(lambda x: corner(x, 4))

In [126]:
horse_result[horse_result['final_corner'].isnull()].head()

,日付,開催,オッズ,人気,着順,斤量,距離,通過,上り,賞金,time_index,race_park,race_type,course_len,final_corner


In [127]:
horse_result.dropna(subset=["final_corner"], inplace=True)

####　　上り
欠損値削除のみ

In [128]:
horse_result.dropna(subset=["上り"], inplace=True)

#### 賞金
欠損値をもう一度調べる

In [129]:
hoge = horse_result.query('賞金!=賞金',engine='numexpr')

In [130]:
hoge.isnull().sum()

日付              0.0
開催              0.0
オッズ             0.0
人気              0.0
着順              0.0
斤量              0.0
距離              0.0
通過              0.0
上り              0.0
賞金              0.0
time_index      0.0
race_park       0.0
race_type       0.0
course_len      0.0
final_corner    0.0
dtype: float64

これから処理するtime_index以外に欠損値はないので全て0埋めする

In [131]:
horse_result['賞金'].fillna(0, inplace=True)

#### time_index
不自然な空白行を欠損値に変換する

In [139]:
horse_result["time_index"] = horse_result["time_index"].replace([''], np.nan)

In [141]:
horse_result['time_index'].isnull().sum()

16972

In [142]:
hoge = horse_result[horse_result['time_index'].isnull()]

In [143]:
hoge.head(50)

,日付,開催,オッズ,人気,着順,斤量,距離,通過,上り,賞金,time_index,race_park,race_type,course_len,final_corner
2012100539,2019/03/16,2中京3,197.2,14.0,10,60.0,障3000,10-10-11-10,13.6,0.0,NaN,中京,障,30.0,10
2012100539,2019/01/19,1中京1,139.4,14.0,12,60.0,障3000,4-6-7-7,13.6,0.0,NaN,中京,障,30.0,7
2012100539,2018/10/28,3新潟6,124.7,11.0,12,60.0,障2850,13-12-12-12,13.6,0.0,NaN,新潟,障,28.0,12
2012105356,2014/07/01,門別,2.1,1.0,5,54.0,ダ1200,1-2,40.2,17.5,NaN,門別,ダ,12.0,2
2012105356,2014/06/12,門別,1.2,1.0,1,54.0,ダ1000,1-1,37.2,150.0,NaN,門別,ダ,10.0,1
2011106317,2015/03/05,笠松,1.6,1.0,1,56.0,ダ1400,2-1-1,38.7,19.0,NaN,笠松,ダ,14.0,1
2011106317,2015/02/16,笠松,1.2,1.0,1,56.0,ダ1400,1-1-1,38.1,13.0,NaN,笠松,ダ,14.0,1
2011106317,2015/02/05,笠松,2.1,1.0,4,56.0,ダ1400,1-1-1,38.4,0.0,NaN,笠松,ダ,14.0,1
2011106317,2015/01/12,笠松,1.4,1.0,1,56.0,ダ1600,2-2-1,38.3,14.0,NaN,笠松,ダ,16.0,1
2011106317,2014/12/30,笠松,3.5,2.0,1,56.0,ダ1400,1-1-1,37.6,17.0,NaN,笠松,ダ,14.0,1


- 障害競走や地方競走の一部レースの模様
- 今回の分析対象が中央競馬であること、レースで障害競走の占める割合が相対的に少数であることから削除する

In [145]:
horse_result.dropna(subset=["time_index"], inplace=True)

In [146]:
horse_result["time_index"].value_counts()

51                                                                                                                                                9380
82                                                                                                                                                9369
83                                                                                                                                                9326
81                                                                                                                                                9306
86                                                                                                                                                9258
79                                                                                                                                                9232
84                                                                                            

まだ不自然な行が残っているので強制的に変換する

In [148]:
horse_result["time_index"] = horse_result["time_index"].replace(['**'], np.nan)

In [149]:
hoge = horse_result[horse_result['time_index'].isnull()]

In [151]:
horse_result['time_index'].isnull().sum()

45

In [152]:
horse_result["time_index"] = pd.to_numeric(horse_result["time_index"], errors="coerce")

In [154]:
horse_result.dropna(subset=["time_index"], inplace=True)

### データ型修正

In [172]:
horse_result['人気'] = horse_result['人気'].astype(int)
horse_result['着順'] = horse_result['着順'].astype(int)
horse_result['time_index'] = horse_result['time_index'].astype(int)
horse_result['course_len'] = horse_result['course_len'].astype(int)
horse_result['賞金'] = horse_result['賞金'].astype(int)

一部列削除

In [174]:
horse_result.drop(['開催','距離','通過'],inplace=True,axis=1)

カラム名変更

In [284]:
# カラム名変換
horse_result.rename(columns={"日付":"date","斤量":"weight_j" ,"オッズ": "odds",
                   "人気": "favorite","着順": "rank","上り":"last","賞金":"money"}, inplace=True)

### 再度全体把握

In [285]:
horse_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 741492 entries, 2013102966 to 2017105663
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   date          741492 non-null  object 
 1   odds          741492 non-null  float64
 2   favorite      741492 non-null  int64  
 3   rank          741492 non-null  int64  
 4   weight_j      741492 non-null  float64
 5   last          741492 non-null  float64
 6   money         741492 non-null  int64  
 7   time_index    741492 non-null  int64  
 8   race_park     741492 non-null  object 
 9   race_type     741492 non-null  object 
 10  course_len    741492 non-null  int64  
 11  final_corner  741492 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 73.5+ MB


In [176]:
horse_result.head()

,日付,オッズ,人気,着順,斤量,上り,賞金,time_index,race_park,race_type,course_len,final_corner
2013102966,2017/01/26,101.9,15,16,56.0,42.0,0,26,大井,ダ,16,16
2013102966,2017/01/12,116.1,7,8,56.0,41.3,0,12,船橋,ダ,16,8
2013102966,2016/12/29,183.6,16,16,56.0,42.4,0,-8,大井,ダ,12,16
2013102966,2016/12/15,88.3,9,8,56.0,40.5,0,60,川崎,ダ,15,9
2013102966,2016/11/29,80.0,11,14,56.0,45.5,0,4,大井,ダ,16,10


実際の分析の際には加工前の列は削除する

## 1.3　血統データ

In [159]:
peds = pd.read_pickle('data/peds_all.pickle')

### 全体把握

In [160]:
peds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56305 entries, 2013102966 to 2019101689
Data columns (total 62 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   peds_0   56305 non-null  object
 1   peds_1   56304 non-null  object
 2   peds_2   56302 non-null  object
 3   peds_3   56304 non-null  object
 4   peds_4   56297 non-null  object
 5   peds_5   56302 non-null  object
 6   peds_6   56304 non-null  object
 7   peds_7   56304 non-null  object
 8   peds_8   56304 non-null  object
 9   peds_9   56296 non-null  object
 10  peds_10  56302 non-null  object
 11  peds_11  56304 non-null  object
 12  peds_12  56302 non-null  object
 13  peds_13  56304 non-null  object
 14  peds_14  56304 non-null  object
 15  peds_15  56304 non-null  object
 16  peds_16  56304 non-null  object
 17  peds_17  56304 non-null  object
 18  peds_18  56296 non-null  object
 19  peds_19  56302 non-null  object
 20  peds_20  56302 non-null  object
 21  peds_21  56302 non-null  o

In [161]:
peds.head()

,peds_0,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,peds_10,peds_11,peds_12,peds_13,peds_14,peds_15,peds_16,peds_17,peds_18,peds_19,peds_20,peds_21,peds_22,peds_23,peds_24,peds_25,peds_26,peds_27,peds_28,peds_29,peds_30,peds_31,peds_32,peds_33,peds_34,peds_35,peds_36,peds_37,peds_38,peds_39,peds_40,peds_41,peds_42,peds_43,peds_44,peds_45,peds_46,peds_47,peds_48,peds_49,peds_50,peds_51,peds_52,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61
2013102966,サムライハート 2002 鹿毛 [血統][産駒] Halo系,アイノサウスポー 2000 鹿毛 [血統][産駒] FNo.[3-l],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],エアグルーヴ 1993 鹿毛 [血統][産駒],スキャン Scan(米) 1988 鹿毛 [血統][産駒],プリティージャナー 1984 芦毛 [血統][産駒],Halo 1969 [血統][産駒],Wishing Well 1975 [血統][産駒],トニービン Tony Bin(愛) 1983 鹿毛 [血統][産駒],ダイナカール 1980 鹿毛 [血統][産駒],Mr. Prospector 1970 鹿毛 [血統][産駒],Video 1983 [血統][産駒],ゼダーン Zeddaan(英) 1965 芦毛 [血統][産駒],プロントモアー 1976 黒鹿毛 [血統][産駒],Hail to Reason 1958 黒鹿毛,Cosmah 1953,Understanding 1963,Mountain Flower 1964,カンパラ 1976 黒鹿毛,Severn Bridge 1965,ノーザンテースト 1971 栗毛,シャダイフェザー 1973 鹿毛,Raise a Native 1961 栗毛,Gold Digger 1962 鹿毛,Nijinsky 1967 鹿毛,Foreseer 1969,Grey Sovereign 1948 芦毛,Vareta 1953,プロント 1963 鹿毛,トサモアー弐世 1959 鹿毛,Turn-to,Nothirdchance,Cosmic Bomb,Almahmoud,Promised Land,Pretty Ways,Montparnasse,Edelweiss,Kalamoun,State Pension,Hornbeam,Priddy Fair,Northern Dancer,Lady Victoria,ガーサント,パロクサイド,Native Dancer,Raise You,Nashua,Sequence,Northern Dancer,Flaming Page,Round Table,Regal Gleam,Nasrullah,Kong,Vilmorin,Veronique,Prince Taj,La Caravelle,トシシロ,トサモアー
2013106468,ジャングルポケット 1998 鹿毛 [血統][産駒] ゼダーン系,システィンチャペル 2000 鹿毛 [血統][産駒] FNo.[5-h],トニービン Tony Bin(愛) 1983 鹿毛 [血統][産駒],ダンスチャーマー Dance Charmer(米) 1990 黒鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],サドラーズギャル Saddlers Gal(愛) 1989 鹿毛 [血統][産駒],カンパラ Kampala(英) 1976 黒鹿毛 [血統][産駒],Severn Bridge 1965 [血統][産駒],Nureyev 1977 鹿毛 [血統][産駒],Skillful Joy 1979 [血統][産駒],Halo 1969 [血統][産駒],Wishing Well 1975 [血統][産駒],Sadler's Wells (米) 1981 鹿毛 [血統][産駒],Glenveagh 1986 [血統][産駒],Kalamoun 1970 芦毛,State Pension 1967,Hornbeam 1953,Priddy Fair 1956,Northern Dancer 1961 鹿毛,Special 1969,Nodouble 1965,Skillful Miss 1974,Hail to Reason 1958 黒鹿毛,Cosmah 1953,Understanding 1963,Mountain Flower 1964,Northern Dancer 1961 鹿毛,Fairy Bridge 1975 鹿毛,Seattle Slew 1974 黒鹿毛,Lisadell 1971,ゼダーン,Khairunissa,オンリーフォアライフ,Lorelei,Hyperion,Thicket,Preciptic,Campanette,Nearctic,Natalma,Forli,Thong,Noholme,Abla-Jay,Daryl's Joy,Poliniss,Turn-to,Nothirdchance,Cosmic Bomb,Almahmoud,Promised Land,Pretty Ways,Montparnasse,Edelweiss,Nearctic,Natalma,Bold Reason,Special,Bold Reasoning,My Charmer,Forli,Thong
2013105712,ファルブラヴ Falbrav(愛) 1998 鹿毛 [血統][産駒] Northern Da...,リトミックダンス 2001 鹿毛 [血統][産駒] FNo.[9-c],Fairy King 1982 [血統][産駒],Gift of the Night 1990 [血統][産駒],ダンスインザダーク 1993 鹿毛 [血統][産駒],レガシーオブストレングス Legacy of Strength(米) 1982 栗毛 [血統...,Northern Dancer 1961 鹿毛 [血統][産駒],Fairy Bridge 1975 鹿毛 [血統][産駒],Slewpy 1980 [血統][産駒],Little Nana 1975 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ダンシングキイ 1983 鹿毛 [血統][産駒],Affirmed 1975 [血統][産駒],Katonka 1972 [血統][産駒],Nearctic 1954 黒鹿毛,Natalma 1957 鹿毛,Bold Reason 1968 鹿毛,Special 1969,Seattle Slew 1974 黒鹿毛,Rare Bouquet 1963,Lithiot 1965,Nenana Road 1970,Halo 1969,Wishing Well 1975,Nijinsky 1967 鹿毛,Key Partner 1976,Exclusive Native 1965,Won't Tell You 1962,Minnesota Mac 1964,Minnetonka 1967,Nearco,Lady Angela,Native Dancer,Almahmoud,Hail to Reason,Lalun,Forli,Thong,Bold Reasoning,My Charmer,Prince John,Forest Song,Ribot,Lithia,Kirkland Lake,Sena,Hail to Reason,Cosmah,Understanding,Mountain Flower,Northern Dancer,Flaming Page,Key to the Mint,Native Partner,Raise a Native,Exclusive,Crafty Admiral,Scarlet Ribbon,Rough'n Tumble,Cow Girl,Chieftain,Heliolight
2013106165,ハービンジャー Harbinger(英) 2006 鹿毛 [血統][産駒] Danzig系,ルミナスポイント 2003 青鹿毛 [血統][産駒] FNo.[B3],Dansili 1996 黒鹿毛 [血統][産駒],Penang Pearl 1996 [血統][産駒],アグネスタキオン 1998 栗毛 [血統][産駒],ソニンク Soninke(英) 1996 黒鹿毛 [血統][産駒],デインヒル Danehill(米) 1986 鹿毛 [血統][産駒],Hasili 1991 [血統][産駒],Bering 1983 栗毛 [血統][産駒],Guapa 198

使用するのは父親（peds_0）と母の父親（peds_4）のみ

## 1.4　騎手データ

In [177]:
jr = pd.read_pickle('data/jockey_results.pickle')

### 全体把握

In [179]:
jr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4381 entries, 01019 to 01186
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   year        4381 non-null   datetime64[ns]
 1   win_rate_1  4381 non-null   float64       
 2   win_rate_3  4381 non-null   float64       
 3   prize       4381 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 171.1+ KB


In [180]:
jr.head()

jockey_id,year,win_rate_1,win_rate_3,prize
01019,2021-12-31,0.029,0.162,15575.5
01019,2020-12-31,0.062,0.233,50873.6
01019,2019-12-31,0.070,0.246,73070.6
01019,2018-12-31,0.063,0.204,76656.6
01019,2017-12-31,0.080,0.227,79820.1


既に加工済み

## 1.5　調教師データ

In [183]:
tr = pd.read_pickle('data/trainer_results.pickle')

### 全体把握

In [184]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6870 entries, 00379 to 01182
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   year        6870 non-null   datetime64[ns]
 1   win_rate_1  6870 non-null   float64       
 2   win_rate_3  6870 non-null   float64       
 3   prize       6870 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 268.4+ KB


In [185]:
tr.head()

trainer_id,year,win_rate_1,win_rate_3,prize
00379,2018-12-31,0.022,0.044,2260.6
00379,2017-12-31,0.045,0.104,14247.5
00379,2016-12-31,0.028,0.118,12531.7
00379,2015-12-31,0.026,0.140,22415.6
00379,2014-12-31,0.060,0.138,25628.8


同じく加工済み

## 1.6.予測対象出馬表データ

### スクレイピング用関数

In [194]:
def scrape(race_id_list, date):
        data = pd.DataFrame()
        for race_id in tqdm(race_id_list):

            url = "https://race.netkeiba.com/race/shutuba.html?race_id=" + race_id
            df = pd.read_html(url)[0]
            df = df.T.reset_index(level=0, drop=True).T

            html = requests.get(url)
            html.encodeing = "EUC-JP"
            soup = BeautifulSoup(html.content, "html.parser")
            race_number = re.findall(
                r'\d+', str(soup.find("span", attrs={"class": "RaceNum"})))
            df["race_number"] = race_number * len(df)
            race_name = soup.find("div", attrs={"class": "RaceName"}).text
            df["race_name"] = [race_name.replace('\n', '')] * len(df)
            texts = soup.find("div", attrs={"class": "RaceData01"}).text
            texts = re.findall(r"\w+", texts)
            for text in texts:
                if "m" in text:
                    # len(df)をかけることでデータフレームの長さだけ内容を追加する
                    df["course_len"] = [
                        int(re.findall(r"\d+", text)[0])] * len(df)
                if text in ["右", "左"]:
                    df["race_turn"] = [text] * len(df)
                if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                    df["weather"] = [text] * len(df)
                if text in ["良", "稍重", "重"]:
                    df["race_condition"] = [text] * len(df)
                if "不" in text:
                    df["race_condition"] = ["不良"] * len(df)
                if '稍' in text:
                    df["race_condition"] = ['稍重'] * len(df)

                if "芝" in text:
                    df["race_type"] = ["芝"] * len(df)
                if "ダ" in text:
                    df["race_type"] = ["ダ"] * len(df)
                if "障" in text:
                    df["race_type"] = ["障害"] * len(df)
                if text in ["札幌", "函館", "福島", "中山", "東京", "新潟", "中京", "京都", "阪神", "小倉"]:
                    df["race_park"] = [text] * len(df)
            texts_2 = soup.find("div", attrs={"class": "RaceData02"}).text
            texts_2 = re.findall(r"\w+", texts_2)
            for text in texts_2:
                if text in ["札幌", "函館", "福島", "中山", "東京", "新潟", "中京", "京都", "阪神", "小倉"]:
                    df["race_park"] = [text] * len(df)
                df["prize"] = [texts_2[texts_2.index("本賞金")+1]] * len(df)
                df["prize"] = df["prize"].astype(int)

            df["date"] = [date] * len(df)
            

            horse_id_list = []
            horse_td_list = soup.find_all("td", attrs={"class": "HorseInfo"})
            for td in horse_td_list:
                horse_id = re.findall(r"\d+", td.find("a")["href"])[0]
                horse_id_list.append(horse_id)

            jockey_id_list = []
            jockey_td_list = soup.find_all("td", attrs={"class": "Jockey"})
            for td in jockey_td_list:
                jockey_id = re.findall(r"\d+", td.find("a")["href"])[0]
                jockey_id_list.append(jockey_id)

            trainer_id_list = []
            trainer_td_list = soup.find_all("td", attrs={"class": "Trainer"})
            for td in trainer_td_list:
                trainer_id = re.findall(r"\d+", td.find("a")["href"])[0]
                trainer_id_list.append(trainer_id)

            df["horse_id"] = horse_id_list
            df["jockey_id"] = jockey_id_list
            df["trainer_id"] = trainer_id_list

            df.index = [race_id] * len(df)
            df['n_horses'] = df.index.map(df.index.value_counts())
            data = data.append(df)
            time.sleep(1)
        return data

In [190]:
race_id_list_n = ['2021040406{}'.format(str(i).zfill(2)) for i in range(1, 13, 1)]
race_id_list_s = ['2021010206{}'.format(str(i).zfill(2)) for i in range(1, 13, 1)]
race_id_list_k = ['2021100406{}'.format(str(i).zfill(2)) for i in range(1, 13, 1)]

In [195]:
#スクレイピング対象
race_id_list = race_id_list_n + race_id_list_s + race_id_list_k 
date = "2021/8/29"

In [196]:
data = scrape(race_id_list, date)

  0%|          | 0/36 [00:00<?, ?it/s]

## 全体把握

In [209]:
confirm_data(data)

<class 'pandas.core.frame.DataFrame'>
Index: 505 entries, 202104040601 to 202110040612
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   枠                   505 non-null    object
 1   馬番                  505 non-null    object
 2   印                   5 non-null      object
 3   馬名                  505 non-null    object
 4   性齢                  505 non-null    object
 5   斤量                  505 non-null    object
 6   騎手                  505 non-null    object
 7   厩舎                  505 non-null    object
 8   馬体重(増減)             505 non-null    object
 9   Unnamed: 9_level_1  505 non-null    object
 10  人気                  505 non-null    object
 11  登録                  0 non-null      object
 12  メモ                  0 non-null      object
 13  race_number         505 non-null    object
 14  race_name           505 non-null    object
 15  course_len          505 non-null    int64 
 16  race_type  

None

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn
202104040601,1,1,NaN,トミケンボハテル,牡7,60.0,草野,美浦古賀史,508(+6),---.-,**,NaN,NaN,1,3歳以上障害未勝利,2850,芝,晴,良,790,新潟,2021/8/29,2014102449,01113,00392,12,NaN
202104040601,2,2,NaN,アームストロング,牡3,58.0,熊沢,美浦堀井,496(+2),---.-,**,NaN,NaN,1,3歳以上障害未勝利,2850,芝,晴,良,790,新潟,2021/8/29,2018100905,00652,00436,12,NaN
202104040601,3,3,NaN,セーリング,牡4,60.0,金子,美浦和田雄,452(+4),---.-,**,NaN,NaN,1,3歳以上障害未勝利,2850,芝,晴,良,790,新潟,2021/8/29,2017106637,01052,01143,12,NaN
202104040601,4,4,NaN,ルレーヴドゥリリ,牝4,58.0,上野,美浦田中剛,488(-2),---.-,**,NaN,NaN,1,3歳以上障害未勝利,2850,芝,晴,良,790,新潟,2021/8/29,2017103710,01087,01119,12,NaN
202104040601,5,5,NaN,ゲインスプレマシー,セ5,60.0,石神,美浦高柳瑞,486(+6),---.-,**,NaN,NaN,1,3歳以上障害未勝利,2850,芝,晴,良,790,新潟,2021/8/29,2016104517,01059,01118,12,NaN


## 個数の把握・欠損値の確認

In [211]:
confirm_series(data,'枠')

8    76
7    74
6    68
5    67
3    60
4    60
2    52
1    48
Name: 枠, dtype: int64

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn


一部の列のみ確認する

In [212]:
confirm_series(data,'馬番')

1     36
2     36
3     36
4     36
5     36
6     36
7     36
8     36
9     35
10    34
11    32
12    31
13    24
14    24
15    16
16    12
17     5
18     4
Name: 馬番, dtype: int64

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn


In [213]:
confirm_series(data,'馬番')

1     36
2     36
3     36
4     36
5     36
6     36
7     36
8     36
9     35
10    34
11    32
12    31
13    24
14    24
15    16
16    12
17     5
18     4
Name: 馬番, dtype: int64

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn


In [214]:
confirm_series(data,'性齢')

牝3    132
牡3     94
牡2     65
牝2     52
牝4     38
牡4     29
牡5     22
牝5     17
牡6     14
牡7      8
セ6      7
セ3      6
牝6      6
セ5      6
牡8      4
セ4      2
セ2      1
セ8      1
牝7      1
Name: 性齢, dtype: int64

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn


レースデータと同様に処理

In [216]:
confirm_series(data,'斤量')

54.0    200
56.0     76
52.0     59
57.0     45
55.0     43
51.0     31
53.0     27
60.0      8
58.0      7
49.0      5
50.0      4
Name: 斤量, dtype: int64

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn


In [218]:
confirm_series(data,'course_len')

2000    87
1800    82
1200    81
1700    74
1000    65
1600    44
1500    26
1400    18
2600    16
2850    12
Name: course_len, dtype: int64

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn


In [219]:
confirm_series(data,'race_type')

芝    324
ダ    181
Name: race_type, dtype: int64

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn


In [220]:
confirm_series(data,'weather')

晴    505
Name: weather, dtype: int64

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn


In [221]:
confirm_series(data,'race_condition')

良    505
Name: race_condition, dtype: int64

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn


In [222]:
confirm_series(data,'prize')

510     193
760      91
700      67
1500     58
1060     28
2500     18
4100     16
790      12
3100     12
2400     10
Name: prize, dtype: int64

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn


In [223]:
confirm_series(data,'race_park')

新潟    180
札幌    166
小倉    159
Name: race_park, dtype: int64

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn


In [224]:
confirm_series(data,'n_horses')

14    112
16    112
12     84
18     72
15     60
10     20
17     17
11     11
9       9
8       8
Name: n_horses, dtype: int64

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn


In [225]:
confirm_series(data,'race_turn')

右    325
左    150
Name: race_turn, dtype: int64

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed: 9_level_1,人気,登録,メモ,race_number,race_name,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn
202104040601,1,1,NaN,トミケンボハテル,牡7,60.0,草野,美浦古賀史,508(+6),---.-,**,NaN,NaN,1,3歳以上障害未勝利,2850,芝,晴,良,790,新潟,2021/8/29,2014102449,01113,00392,12,NaN
202104040601,2,2,NaN,アームストロング,牡3,58.0,熊沢,美浦堀井,496(+2),---.-,**,NaN,NaN,1,3歳以上障害未勝利,2850,芝,晴,良,790,新潟,2021/8/29,2018100905,00652,00436,12,NaN
202104040601,3,3,NaN,セーリング,牡4,60.0,金子,美浦和田雄,452(+4),---.-,**,NaN,NaN,1,3歳以上障害未勝利,2850,芝,晴,良,790,新潟,2021/8/29,2017106637,01052,01143,12,NaN
202104040601,4,4,NaN,ルレーヴドゥリリ,牝4,58.0,上野,美浦田中剛,488(-2),---.-,**,NaN,NaN,1,3歳以上障害未勝利,2850,芝,晴,良,790,新潟,2021/8/29,2017103710,01087,01119,12,NaN
202104040601,5,5,NaN,ゲインスプレマシー,セ5,60.0,石神,美浦高柳瑞,486(+6),---.-,**,NaN,NaN,1,3歳以上障害未勝利,2850,芝,晴,良,790,新潟,2021/8/29,2016104517,01059,01118,12,NaN


障害レースでは入らないようだ

## データ修正

In [228]:
data["sex"] = data["性齢"].map(lambda x: str(x)[0])
data["age"] = data["性齢"].map(lambda x: str(x)[1:]).astype(int)

## カラム名変更・データ抽出

In [233]:
data["frame_number"] = data["枠"].astype(int)
data["horse_number"] = data["馬番"].astype(int)
data["weight_j"] = pd.to_numeric(data["斤量"], errors="coerce")
data["weight_j"] = data["weight_j"].astype(float)

data["date"] = pd.to_datetime(data["date"])

data["course_len"] = data["course_len"] // 100

In [278]:
data.drop(["枠","馬番","印","馬名","性齢","斤量","騎手","厩舎","馬体重(増減)","Unnamed: 9_level_1","登録","メモ","人気","race_number","race_name"],axis=1,inplace=True)

In [279]:
confirm_data(data)

<class 'pandas.core.frame.DataFrame'>
Index: 505 entries, 202104040601 to 202110040612
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   course_len      505 non-null    int64         
 1   race_type       505 non-null    object        
 2   weather         505 non-null    object        
 3   race_condition  505 non-null    object        
 4   prize           505 non-null    int64         
 5   race_park       505 non-null    object        
 6   date            505 non-null    datetime64[ns]
 7   horse_id        505 non-null    object        
 8   jockey_id       505 non-null    object        
 9   trainer_id      505 non-null    object        
 10  n_horses        505 non-null    int64         
 11  race_turn       475 non-null    object        
 12  sex             505 non-null    object        
 13  age             505 non-null    int64         
 14  frame_number    505 non-null    int64      

None

,course_len,race_type,weather,race_condition,prize,race_park,date,horse_id,jockey_id,trainer_id,n_horses,race_turn,sex,age,frame_number,horse_number,weight_j
202104040601,28,芝,晴,良,790,新潟,2021-08-29,2014102449,01113,00392,12,NaN,牡,7,1,1,60.0
202104040601,28,芝,晴,良,790,新潟,2021-08-29,2018100905,00652,00436,12,NaN,牡,3,2,2,58.0
202104040601,28,芝,晴,良,790,新潟,2021-08-29,2017106637,01052,01143,12,NaN,牡,4,3,3,60.0
202104040601,28,芝,晴,良,790,新潟,2021-08-29,2017103710,01087,01119,12,NaN,牝,4,4,4,58.0
202104040601,28,芝,晴,良,790,新潟,2021-08-29,2016104517,01059,01118,12,NaN,セ,5,5,5,60.0


## 1.7.データまとめ

### レースデータ


#### 参照元
以下のURLよりスクレピングで入手した,2011年〜2020年までの10年間の
中央競馬のレースデータを加工したもの。

[netkeiba.com](https://www.netkeiba.com/）

#### テーブル定義
|物理名|論理名|値|データ型|分類|
|:--|:--|:--|:--|:--|
|rank|レースの着順|1~18|int64|目的変数|◯|◯|◯|
|frame_number|出走馬を8つのグループに分けたもの|1~8|int64|説明変数||-|-|
|horse_number|出走馬1頭ずつにつけられた番号|1~18|int64|説明変数|◯|◯|◯|
|weight_j|騎手の重さ|任意の値|float64|説明変数|◯|◯|◯|
|odds|単勝オッズ|任意の値|float64|説明変数|◯|◯|◯|
|favorite|単勝オッズの昇順|1~18|int64|説明変数|◯|◯|◯|
|prize|レースで1着になった場合の獲得賞金|任意の値|float64|説明変数|◯|◯|◯|
|race_turn|レースの周回形式|右・左・直・ー・芝の５種類|object|説明変数|◯|◯|◯|
|course_len|レースの距離|1000〜4100|int64|説明変数|◯|◯|◯|
|weather|レース時の天候|晴・曇・雨・雪・小雨・小雪の6種類|object|説明変数|◯|◯|-|
|race_type|レースの種類|芝・ダート・障害の3種類|object|説明変数|-|-|-|
|race_condition|馬場の状態|良・稍重・重・不良の4種類|object|説明変数|-|-|-|
|n_horses|レースの出走頭数|5~18|int64|説明変数|◯|◯|◯|
|date|レース開催日|2011~2020|object|他のデータとの結合・特徴量エンジニアリングで使用|◯|-|-|
|race_park|競馬場名|中央競馬は10種類で地方や海外も含む|object|説明変数|◯|◯|-|
|horse_id|競走馬のid|任意の値|object|他のデータとの結合用|◯|◯|◯|
|jockey_id|騎手のid|任意の値|object|他のデータとの結合用|◯|◯|◯|
|trainer_id|調教師のid|任意の値|object|他のデータとの結合用|◯|◯|◯|
|sex|馬の性別|性別は牡・牝・セの3種類|object|説明変数|◯|◯|◯|
|age|馬の年齢|年齢は2~12|int64|説明変数|◯|◯|◯|
|weihgt|馬の体重|任意の値|int64|説明変数|◯|◯|◯|
|weihgt_j|馬の体重変化|任意の値|int64|説明変数|◯|◯|◯|


#### Number of Instances
440748

#### Number of Attributes
21数値/一番左の列にあるrankを予測する。

In [282]:
confirm_data(race_result)

<class 'pandas.core.frame.DataFrame'>
Index: 440748 entries, 201101010101 to 202004040608
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   rank            440748 non-null  int64         
 1   frame_number    440748 non-null  int64         
 2   horse_number    440748 non-null  int64         
 3   weight_j        440748 non-null  float64       
 4   odds            440748 non-null  float64       
 5   favorite        440748 non-null  int64         
 6   prize           440748 non-null  object        
 7   race_turn       440748 non-null  object        
 8   course_len      440748 non-null  int64         
 9   weather         440748 non-null  object        
 10  race_type       440748 non-null  object        
 11  race_condition  440748 non-null  object        
 12  date            440748 non-null  datetime64[ns]
 13  race_park       440748 non-null  object        
 14  horse_id        440748 n

None

,rank,frame_number,horse_number,weight_j,odds,favorite,prize,race_turn,course_len,weather,race_type,race_condition,date,race_park,horse_id,jockey_id,trainer_id,sex,age,weight,weight_change
201101010101,1,2,2,54.0,4.7,3,500.0,右,1800,晴,芝,良,2011-08-13,札幌,2009105006,01019,00379,牡,2,428,8
201101010101,2,3,3,54.0,3.4,2,500.0,右,1800,晴,芝,良,2011-08-13,札幌,2009100577,05203,01061,牡,2,440,-14
201101010101,3,5,5,54.0,2.1,1,500.0,右,1800,晴,芝,良,2011-08-13,札幌,2009102908,01093,01089,牡,2,456,-6
201101010101,4,4,4,54.0,7.0,4,500.0,右,1800,晴,芝,良,2011-08-13,札幌,2009109159,01095,01075,牡,2,456,0
201101010101,5,1,1,54.0,17.5,5,500.0,右,1800,晴,芝,良,2011-08-13,札幌,2009105263,01127,01026,牝,2,460,-10


### 競走馬データ

#### 参照元
2011年〜2021年までの中央競馬のレースに出走した
競走馬の過去成績データを加工したもの。
次のHPよりスクレイピングした。

[netkeiba.com](https://www.netkeiba.com/）

#### テーブル定義
|物理名|論理名|値|データ型|分類|
|:--|:--|:--|:--|:--|
|date|レース施行日|任意の日付|object|他のデータとの結合用・特徴量エンジニアリングで使用|-|-|-|
|odds|単勝馬券が的中した場合の「倍率」|1から始まる連続値|float64|説明変数|◯|-|-|
|favorite|単勝の階級値|1~18|float64|説明変数|◯|◯|-|
|rank|そのレースの着順|基本的に1~18|object|説明変数|◯|◯|◯|
|weight_j|騎手の重さ|任意の値|float64|説明変数|◯|◯|◯|
|last|対象馬の上がり３ハロンのタイム|任意の値|float64|説明変数|◯|◯|◯|
|money|レース獲得賞金|任意の値|float64|説明変数|◯|◯|◯|
|time_index|競走馬の能力を数値化したもの|任意の値|object|説明変数|◯|-|-|
|race_park|レースの開催場所|東京など|object|説明変数|◯|◯|◯|
|race_type|レースの開催形態|芝・ダート・障害|object|説明変数|◯|◯|-|
|course_len|レースの開催距離|任意の値|int64|説明変数|◯|◯|◯|
|final_corner|4コーナー通過順位|1~18|int64|説明変数|◯|◯|-|


#### Number of Instances
741492

#### Number of Attributes
12数値/レースデータと統合する

In [286]:
confirm_data(horse_result)

<class 'pandas.core.frame.DataFrame'>
Index: 741492 entries, 2013102966 to 2017105663
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   date          741492 non-null  object 
 1   odds          741492 non-null  float64
 2   favorite      741492 non-null  int64  
 3   rank          741492 non-null  int64  
 4   weight_j      741492 non-null  float64
 5   last          741492 non-null  float64
 6   money         741492 non-null  int64  
 7   time_index    741492 non-null  int64  
 8   race_park     741492 non-null  object 
 9   race_type     741492 non-null  object 
 10  course_len    741492 non-null  int64  
 11  final_corner  741492 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 73.5+ MB


None

,date,odds,favorite,rank,weight_j,last,money,time_index,race_park,race_type,course_len,final_corner
2013102966,2017/01/26,101.9,15,16,56.0,42.0,0,26,大井,ダ,16,16
2013102966,2017/01/12,116.1,7,8,56.0,41.3,0,12,船橋,ダ,16,8
2013102966,2016/12/29,183.6,16,16,56.0,42.4,0,-8,大井,ダ,12,16
2013102966,2016/12/15,88.3,9,8,56.0,40.5,0,60,川崎,ダ,15,9
2013102966,2016/11/29,80.0,11,14,56.0,45.5,0,4,大井,ダ,16,10


### 血統データ

#### 参照元
2011年〜2021年までの中央競馬のレースに出走した
競走馬の血統データ。
次のHPよりスクレイピングした。

[netkeiba.com](https://www.netkeiba.com/）

#### テーブル定義
|物理名|論理名|値|データ型|分類|
|:--|:--|:--|:--|:--|
|peds_0|馬の父親|任意の名前|object|説明変数|-|-|-|
|peds_4|馬の母父親|任意の名前|object|説明変数|-|-|-|
|それ以外の値は全て採用しない|


#### Number of Instances
52449

#### Number of Attributes
2数値、Labelencodingした後にcategory型に変換してレースデータと統合する


In [292]:

confirm_data(peds[['peds_0','peds_4']])

<class 'pandas.core.frame.DataFrame'>
Index: 56305 entries, 2013102966 to 2019101689
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   peds_0  56305 non-null  object
 1   peds_4  56297 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


None

,peds_0,peds_4
2013102966,サムライハート 2002 鹿毛 [血統][産駒] Halo系,スキャン Scan(米) 1988 鹿毛 [血統][産駒]
2013106468,ジャングルポケット 1998 鹿毛 [血統][産駒] ゼダーン系,サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒]
2013105712,ファルブラヴ Falbrav(愛) 1998 鹿毛 [血統][産駒] Northern Da...,ダンスインザダーク 1993 鹿毛 [血統][産駒]
2013106165,ハービンジャー Harbinger(英) 2006 鹿毛 [血統][産駒] Danzig系,アグネスタキオン 1998 栗毛 [血統][産駒]
2013104911,ダイワメジャー 2001 栗毛 [血統][産駒] Halo系,ブライアンズタイム Brian's Time(米) 1985 黒鹿毛 [血統][産駒]


### き

#### 参照元
2011年〜2021年までの中央競馬のレースに出走した
騎手の成績データ。
次のHPよりスクレイピングした。

[netkeiba.com](https://www.netkeiba.com/）

#### テーブル定義
|物理名|論理名|値|データ型|分類|
|:--|:--|:--|:--|:--|
|year|出走記録のあるレースの年（後の統合のために12/31としている）|年/12/31|datetime|データ統合時に利用|-|-|-|
|win_rate_1|騎手の勝率|任意の値|float64|説明変数|-|-|-|
|win_rate_3|騎手の複勝率（1~3着以内）|任意の値|float64|説明変数|-|-|-|
|prize|騎手の獲得賞金|任意の値|float64|説明変数|-|-|-|


#### Number of Instances
4381

#### Number of Attributes
4数値、レースデータと統合する


In [294]:

confirm_data(jr)

<class 'pandas.core.frame.DataFrame'>
Index: 4381 entries, 01019 to 01186
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   year        4381 non-null   datetime64[ns]
 1   win_rate_1  4381 non-null   float64       
 2   win_rate_3  4381 non-null   float64       
 3   prize       4381 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 171.1+ KB


None

jockey_id,year,win_rate_1,win_rate_3,prize
01019,2021-12-31,0.029,0.162,15575.5
01019,2020-12-31,0.062,0.233,50873.6
01019,2019-12-31,0.070,0.246,73070.6
01019,2018-12-31,0.063,0.204,76656.6
01019,2017-12-31,0.080,0.227,79820.1


### 調教師データ

#### 参照元
2011年〜2021年までの中央競馬のレースに出走した競走馬を管理している調教師
の成績データ。
次のHPよりスクレイピングした。

[netkeiba.com](https://www.netkeiba.com/）

#### テーブル定義
|物理名|論理名|値|データ型|分類|
|:--|:--|:--|:--|:--|
|year|出走記録のあるレースの年（後の統合のために12/31としている）|年/12/31|datetime|データ統合時に利用|-|-|-|
|win_rate_1|調教師の勝率|任意の値|float64|説明変数|-|-|-|
|win_rate_3|調教師の複勝率（1~3着以内）|任意の値|float64|説明変数|-|-|-|
|prize|調教師の獲得賞金|任意の値|float64|説明変数|-|-|-|


#### Number of Instances
6870

#### Number of Attributes
4数値、レースデータと統合する

In [296]:

confirm_data(tr)

<class 'pandas.core.frame.DataFrame'>
Index: 6870 entries, 00379 to 01182
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   year        6870 non-null   datetime64[ns]
 1   win_rate_1  6870 non-null   float64       
 2   win_rate_3  6870 non-null   float64       
 3   prize       6870 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 268.4+ KB


None

trainer_id,year,win_rate_1,win_rate_3,prize
00379,2018-12-31,0.022,0.044,2260.6
00379,2017-12-31,0.045,0.104,14247.5
00379,2016-12-31,0.028,0.118,12531.7
00379,2015-12-31,0.026,0.140,22415.6
00379,2014-12-31,0.060,0.138,25628.8
